In [ ]:
import urllib.request
from zipfile import ZipFile
import pandas as pd
import os

In [ ]:
import datetime

x = datetime.datetime.now()
etl_date = x.strftime("%Y%m%d_%H%M%S")
print(etl_date) 
%set_env etl_date={etl_date}

In [ ]:
import simplejson
with open('/opt/ml/metadata/resource-metadata.json') as fh:
    metadata = simplejson.loads(fh.read())
accountid = metadata['ResourceArn'].split(':')[4]

%set_env accountid={accountid}
%set_env bucket_name=lab-{accountid}

## Change the Value of the file_name below!

In [ ]:
from pyarrow.parquet import ParquetDataset
import s3fs

folder_name= "s3://" + os.getenv('bucket_name') + "/data/analytics/best_movies/"
s3 = s3fs.S3FileSystem()
paths = [path for path in s3.ls(folder_name) if path.endswith(".parquet.snappy")]
dataset = ParquetDataset(paths, filesystem=s3)
table = dataset.read()
movies_df = table.to_pandas()
print("movies_df has %s lines" % movies_df.shape[0])
movies_df["flag"] = 1
movies_df[0:10]

In [ ]:
movies_df.to_parquet(
        "s3://" + 
        os.getenv('bucket_name') + 
        "/data/analytics/best_movies/best_movies_" +
        etl_date + 
        ".parquet.snappy")

## Now, run you crawler again!